# Grunnkrets investigation

In [20]:
# Magic to automatically update imports if functions in utils are changed
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
from utils import group_df, preprocess_grunnkrets_df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
age_distribution = pd.read_csv("data/grunnkrets_age_distribution.csv")
age_distribution.shape

(22620, 93)

In [22]:
age_distribution.head()

,grunnkrets_id,year,age_0,age_1,age_2,age_3,age_4,age_5,age_6,age_7,...,age_81,age_82,age_83,age_84,age_85,age_86,age_87,age_88,age_89,age_90
0,16013117,2015,14,14,14,14,13,13,12,11,...,0,0,0,0,0,0,0,0,0,0
1,16013117,2016,10,10,10,10,10,10,9,9,...,0,0,0,0,0,0,0,0,0,0
2,11030206,2015,5,5,5,5,5,4,4,3,...,0,0,0,0,0,0,0,0,0,0
3,16011203,2016,2,2,2,2,2,3,3,3,...,1,1,1,1,1,1,1,0,0,0
4,3011601,2016,7,7,7,7,6,6,5,4,...,1,1,0,0,0,0,0,0,0,0


Look at the distribution of stores at different granularities (grunnkrets, delomrade, kommune, fylke) 

In [23]:
from sklearn.model_selection import train_test_split

stores_train = pd.read_csv("data/stores_train.csv")
stores_train, _ = train_test_split(stores_train, test_size=0.2, random_state=0)
stores_train_explore = stores_train[ ['grunnkrets_id'] ].copy()
stores_train_explore["store_count"] = 1
stores_train_explore

,grunnkrets_id,store_count
2506,6020601,1
12305,2350203,1
4142,6240307,1
8561,15040206,1
818,3012312,1
...,...,...
4859,18330207,1
3264,16017522,1
9845,15040101,1
10799,12012008,1


In [24]:
from utils import create_geographical_columns, group_df
stores_train_explore = create_geographical_columns(stores_train_explore)


In [25]:
group_df(stores_train_explore, 'grunnkrets_id')[ ['store_count'] ].value_counts()

store_count
1              1842
2               581
3               285
4               158
5               107
6                79
7                58
8                44
9                40
11               28
10               24
12               19
13               16
14               13
15               11
16               11
18               11
19                8
20                8
17                7
23                6
24                5
21                4
28                4
25                4
32                4
22                3
26                3
29                2
30                2
37                2
27                1
31                1
33                1
35                1
36                1
41                1
42                1
44                1
47                1
51                1
61                1
67                1
dtype: int64

This is very sparsely populated 

In [26]:
group_df(stores_train_explore, 'delomrade')[ ['store_count'] ].value_counts()

store_count
1              269
2              192
3              133
4              114
5               76
              ... 
48               1
27               1
42               1
40               1
205              1
Length: 65, dtype: int64

We see that each delomrade is also quite sparsely populated, except for a few

In [27]:
group_df(stores_train_explore, 'kommune')[ ['store_count'] ].value_counts()

store_count
4              39
1              30
5              29
3              26
8              26
               ..
53              1
52              1
51              1
50              1
1481            1
Length: 76, dtype: int64

We see that most "kommuner" contain few stores

In [28]:
group_df(stores_train_explore, 'fylke')[ ['store_count'] ].value_counts()

store_count
213            1
505            1
944            1
860            1
854            1
592            1
586            1
581            1
573            1
462            1
241            1
414            1
379            1
364            1
360            1
345            1
275            1
258            1
1481           1
dtype: int64

We can see that each fylke contains many stores

IDEAS: group by grunnkrets_id, first 2 digits is "fylke", first 4 corresponds to "kommune" (see https://no.wikipedia.org/wiki/Grunnkretser_i_Norge). First 6 digits are also related in some sense.

In [29]:
age_distribution = preprocess_grunnkrets_df(age_distribution, "grunnkrets_id")

In [30]:
age_list = []
for col in list(age_distribution.columns):
    if col.startswith("age"):
        _, age_num = col.split("_")
        age_list.append([int(age_num), col])
age_list = sorted(age_list, key=lambda x: (x[0]))
print(age_list)

[[0, 'age_0'], [1, 'age_1'], [2, 'age_2'], [3, 'age_3'], [4, 'age_4'], [5, 'age_5'], [6, 'age_6'], [7, 'age_7'], [8, 'age_8'], [9, 'age_9'], [10, 'age_10'], [11, 'age_11'], [12, 'age_12'], [13, 'age_13'], [14, 'age_14'], [15, 'age_15'], [16, 'age_16'], [17, 'age_17'], [18, 'age_18'], [19, 'age_19'], [20, 'age_20'], [21, 'age_21'], [22, 'age_22'], [23, 'age_23'], [24, 'age_24'], [25, 'age_25'], [26, 'age_26'], [27, 'age_27'], [28, 'age_28'], [29, 'age_29'], [30, 'age_30'], [31, 'age_31'], [32, 'age_32'], [33, 'age_33'], [34, 'age_34'], [35, 'age_35'], [36, 'age_36'], [37, 'age_37'], [38, 'age_38'], [39, 'age_39'], [40, 'age_40'], [41, 'age_41'], [42, 'age_42'], [43, 'age_43'], [44, 'age_44'], [45, 'age_45'], [46, 'age_46'], [47, 'age_47'], [48, 'age_48'], [49, 'age_49'], [50, 'age_50'], [51, 'age_51'], [52, 'age_52'], [53, 'age_53'], [54, 'age_54'], [55, 'age_55'], [56, 'age_56'], [57, 'age_57'], [58, 'age_58'], [59, 'age_59'], [60, 'age_60'], [61, 'age_61'], [62, 'age_62'], [63, 'age_6

In [125]:
from utils import group_age_columns


new_age_distribution = group_age_columns(age_distribution_df=age_distribution, span_size=7)

In [126]:
new_age_distribution.head()

,grunnkrets_id,year,fylke,kommune,delomrade,age_0-6,age_7-13,age_14-20,age_21-27,age_28-34,age_35-41,age_42-48,age_49-55,age_56-62,age_63-69,age_70-76,age_77-83,age_84-90
6784,2190812,2015,2,219,21908,0,0,0,3,16,21,18,14,7,1,0,0,0
8175,2190914,2015,2,219,21909,29,54,46,35,23,32,44,47,33,22,12,7,5
8529,20300310,2015,20,2030,203003,2,0,6,16,24,11,7,7,7,0,0,0,0
8577,5170101,2015,5,517,51701,2,0,5,19,32,14,1,0,3,0,0,0,0
16348,2191013,2015,2,219,21910,71,70,36,44,63,76,67,50,47,37,28,25,9


This allows us to group by either "fylke", "kommune", "delomrade" or "grunnkrets"

We can now do this for all grunnkrets-related dataframes

In [153]:
from utils import join_grouped_df, age_bins

full_population_df = create_geographical_columns(stores_train)

full_population_df = join_grouped_df(full_population_df, new_age_distribution[age_bins(age_list, span_size=7) + ['grunnkrets_id']], 'grunnkrets_id')
full_population_df = join_grouped_df(full_population_df, new_age_distribution[age_bins(age_list, span_size=7) + ['delomrade']], 'delomrade')
full_population_df = join_grouped_df(full_population_df, new_age_distribution[age_bins(age_list, span_size=7) + ['kommune']], 'kommune')
full_population_df = join_grouped_df(full_population_df, new_age_distribution[age_bins(age_list, span_size=7) + ['fylke']], 'fylke')


full_population_df.columns

Index(['store_id', 'year', 'store_name', 'plaace_hierarchy_id',
       'sales_channel_name', 'grunnkrets_id', 'address', 'lat', 'lon',
       'chain_name', 'mall_name', 'revenue', 'fylke', 'kommune', 'delomrade',
       'grunnkrets_id.age_0-6', 'grunnkrets_id.age_7-13',
       'grunnkrets_id.age_14-20', 'grunnkrets_id.age_21-27',
       'grunnkrets_id.age_28-34', 'grunnkrets_id.age_35-41',
       'grunnkrets_id.age_42-48', 'grunnkrets_id.age_49-55',
       'grunnkrets_id.age_56-62', 'grunnkrets_id.age_63-69',
       'grunnkrets_id.age_70-76', 'grunnkrets_id.age_77-83',
       'grunnkrets_id.age_84-90', 'delomrade.age_0-6', 'delomrade.age_7-13',
       'delomrade.age_14-20', 'delomrade.age_21-27', 'delomrade.age_28-34',
       'delomrade.age_35-41', 'delomrade.age_42-48', 'delomrade.age_49-55',
       'delomrade.age_56-62', 'delomrade.age_63-69', 'delomrade.age_70-76',
       'delomrade.age_77-83', 'delomrade.age_84-90', 'kommune.age_0-6',
       'kommune.age_7-13', 'kommune.age_14-20',

### Mean and median age

In [154]:
from utils import merge_columns_mean
from utils import merge_age_columns_mean

new_age_distribution2 = pd.read_csv("data/grunnkrets_age_distribution.csv")
new_age_distribution2 = preprocess_grunnkrets_df(new_age_distribution2)

In [155]:
mean_ages_grunnkrets = merge_age_columns_mean(new_age_distribution2, 'grunnkrets_id')
mean_ages_grunnkrets.describe()

,grunnkrets_id.mean_age
count,11379.000000
mean,40.058491
std,5.000218
min,21.694444
25%,36.874665
50%,39.684000
75%,42.807146
max,81.190476


In [156]:
mean_ages_delomrade = merge_age_columns_mean(new_age_distribution2, 'delomrade')
mean_ages_delomrade.describe()

,delomrade.mean_age
count,1541.000000
mean,40.195220
std,3.203201
min,29.243990
25%,37.966243
50%,40.102068
75%,42.268260
max,54.132353


In [157]:
mean_ages_kommune = merge_age_columns_mean(new_age_distribution2, 'kommune')
mean_ages_kommune.describe()

,kommune.mean_age
count,428.000000
mean,40.860425
std,2.415133
min,33.826465
25%,39.101782
50%,41.044260
75%,42.622069
max,47.307203


In [158]:
mean_ages_fylke = merge_age_columns_mean(new_age_distribution2, 'fylke')
mean_ages_fylke.describe()

,fylke.mean_age
count,19.000000
mean,39.530080
std,1.494322
min,36.713186
25%,38.424188
50%,39.936194
75%,40.508635
max,42.140772


In [159]:
full_population_df=full_population_df.merge(mean_ages_grunnkrets, how='left', right_index=True, left_on='grunnkrets_id')
full_population_df=full_population_df.merge(mean_ages_delomrade, how='left', right_index=True, left_on='delomrade')
full_population_df=full_population_df.merge(mean_ages_kommune, how='left', right_index=True, left_on='kommune')
full_population_df=full_population_df.merge(mean_ages_fylke, how='left', right_index=True, left_on='fylke')
full_population_df

,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,...,fylke.age_49-55,fylke.age_56-62,fylke.age_63-69,fylke.age_70-76,fylke.age_77-83,fylke.age_84-90,grunnkrets_id.mean_age,delomrade.mean_age,kommune.mean_age,fylke.mean_age
2506,990857423-990974489-88185,2016,BAKER KLAUSEN UNION,1.2.2.0,Bakery shop,6020601,GRØNLAND 54,59.743135,10.194627,BAKER KLAUSEN DRAMMEN,...,25832,22890,22511,16409,10207,5207,48.224195,38.487914,39.387326,39.956210
12305,948879220-972145203-18800,2016,PRIKKEN RENS AS,3.4.2.0,Laundromats and dry cleaners,2350203,ENERGIVEGEN 5,60.149288,11.187290,NaN,...,58307,46609,42317,31350,19265,9502,40.716040,39.072131,36.761506,38.502433
4142,951136751-971668970-6795,2016,EXPERT HOKKSUND,2.3.1.0,Electronics stores,6240307,SUNDMOEN NÆRINGSOMRÅDE,59.768986,9.899066,EXPERT,...,25832,22890,22511,16409,10207,5207,39.632299,36.351346,39.450117,39.956210
8561,980237508-980246167-71346,2016,INVIT AS,2.9.1.0,Gifts and interior design shops,15040206,APOTEKERGATA 9,62.471607,6.151301,NaN,...,23549,22523,21321,15162,10762,5817,39.373817,40.416574,38.632862,39.936194
818,974421305-974489902-136560,2016,MUCHO MAS,1.1.6.2,Restaurants,3012312,THORVALD MEYERS GATE 36,59.924205,10.759401,NaN,...,52888,43642,38288,26516,17362,9386,31.388585,33.151215,36.713186,36.713186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4859,916746350-916783280-822890,2016,ILONA AS,2.4.6.0,Clothing stores,18330207,FRIDTJOF NANSENS GATE 12,66.310151,14.136919,NaN,...,21702,20551,19447,14469,9828,4855,54.808383,44.494482,40.416473,40.515116
3264,917197334-971789727-13321,2016,FARGERIKE REINAAS,2.1.2.0,Paint dealers,16017522,HEIMDALSVEGEN 12,63.348995,10.355104,FARGERIKE,...,26791,23919,22203,16158,10668,5359,47.122257,37.547427,37.373347,38.345943
9845,916878184-916892322-826818,2016,NEXT FRISØR ÅLESUND,3.2.1.0,Hairdressers,15040101,SKARBØVIKGATA 26,62.466770,6.113498,NaN,...,23549,22523,21321,15162,10762,5817,42.818966,37.996804,38.632862,39.936194
10799,898250172-998266254-472659,2016,HELSE OG VELVÆRE,3.2.4.0,Spas,12012008,KRINGSJÅVEIEN 83,60.389335,5.286455,NaN,...,44135,39021,36476,26081,17591,9404,33.653905,37.714357,38.014037,38.153120


## Look at households

In [17]:

num_persons_df = pd.read_csv("data/grunnkrets_households_num_persons.csv")
num_persons_df.shape

(26009, 10)

In [18]:
num_persons_df.head()

,grunnkrets_id,year,couple_children_0_to_5_years,couple_children_18_or_above,couple_children_6_to_17_years,couple_without_children,single_parent_children_0_to_5_years,single_parent_children_18_or_above,single_parent_children_6_to_17_years,singles
0,12010509,2015,0,0,4,0,0,0,0,4
1,14390206,2015,0,7,0,0,0,0,0,3
2,18040514,2015,10,0,0,0,0,0,0,7
3,18040804,2015,0,0,0,0,0,0,0,0
4,18200101,2015,0,6,0,0,0,0,0,4


In [160]:

num_persons_df.describe()

,grunnkrets_id,year,couple_children_0_to_5_years,couple_children_18_or_above,couple_children_6_to_17_years,couple_without_children,single_parent_children_0_to_5_years,single_parent_children_18_or_above,single_parent_children_6_to_17_years,singles
count,2.600900e+04,26009.000000,26009.000000,26009.000000,26009.000000,26009.000000,26009.000000,26009.000000,26009.000000,26009.000000
mean,1.002896e+07,2015.500019,74.124149,29.244915,80.865854,88.010689,6.190396,10.003960,17.490061,80.364335
std,5.929462e+06,0.500010,96.273040,33.258452,98.161437,92.247059,10.911467,12.710166,23.411638,107.537268
min,1.010102e+06,2015.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.320205e+06,2015.000000,17.000000,9.000000,21.000000,28.000000,0.000000,0.000000,0.000000,20.000000
50%,1.018011e+07,2016.000000,43.000000,20.000000,49.000000,58.000000,0.000000,7.000000,10.000000,44.000000
75%,1.563030e+07,2016.000000,94.000000,38.000000,104.000000,116.000000,9.000000,14.000000,24.000000,98.000000
max,2.030050e+07,2016.000000,1729.000000,497.000000,1613.000000,1066.000000,173.000000,173.000000,309.000000,1700.000000


## Preprocessing ALL the dataframes :)

In [49]:
age_distribution = pd.read_csv("data/grunnkrets_age_distribution.csv")
households_num_persons = pd.read_csv("data/grunnkrets_households_num_persons.csv")
income_households = pd.read_csv("data/grunnkrets_income_households.csv")
norway_stripped = pd.read_csv("data/grunnkrets_norway_stripped.csv")


In [50]:
# add all dataframes to dict
grunnkrets_dfs = {
    "age_distribution" : age_distribution, 
    "households_num_persons" : households_num_persons, 
    "income_households" : income_households, 
    "norway_stripped" : norway_stripped, 
    }

In [51]:
# preprocess dataframes
for df_name, df in grunnkrets_dfs.items():
    grunnkrets_dfs[df_name] = preprocess_grunnkrets_df(df)

# group age categories and sum up values
# span_size determines how many age-columns should be grouped each time
span_size = 7 # I chose 7 because it is a prime factor of 91, and makes all spans have same size
grunnkrets_dfs["age_distribution"] = group_age_columns(age_distribution_df=grunnkrets_dfs["age_distribution"], span_size=span_size)

In [52]:
# add dataframes grouped by fylke, kommune and delomrade
for df_name, df in grunnkrets_dfs.copy().items():
    grunnkrets_dfs[df_name + "_fylke"] = group_df(df, "fylke")
    grunnkrets_dfs[df_name + "_kommune"] = group_df(df, "kommune")
    grunnkrets_dfs[df_name + "_delomrade"] = group_df(df, "delomrade")


In [53]:
grunnkrets_dfs.keys()

dict_keys(['age_distribution', 'households_num_persons', 'income_households', 'norway_stripped', 'age_distribution_fylke', 'age_distribution_kommune', 'age_distribution_delomrade', 'households_num_persons_fylke', 'households_num_persons_kommune', 'households_num_persons_delomrade', 'income_households_fylke', 'income_households_kommune', 'income_households_delomrade', 'norway_stripped_fylke', 'norway_stripped_kommune', 'norway_stripped_delomrade'])

In [54]:
grunnkrets_dfs["age_distribution"]

,grunnkrets_id,year,fylke,kommune,delomrade,age_0-6,age_7-13,age_14-20,age_21-27,age_28-34,age_35-41,age_42-48,age_49-55,age_56-62,age_63-69,age_70-76,age_77-83,age_84-90
6784,2190812,2015,2,219,21908,0,0,0,3,16,21,18,14,7,1,0,0,0
8175,2190914,2015,2,219,21909,29,54,46,35,23,32,44,47,33,22,12,7,5
8529,20300310,2015,20,2030,203003,2,0,6,16,24,11,7,7,7,0,0,0,0
8577,5170101,2015,5,517,51701,2,0,5,19,32,14,1,0,3,0,0,0,0
16348,2191013,2015,2,219,21910,71,70,36,44,63,76,67,50,47,37,28,25,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10062,1061804,2016,1,106,10618,28,26,25,42,33,31,41,49,48,56,29,8,9
10064,1063202,2016,1,106,10632,28,36,35,33,28,38,42,39,35,27,18,11,5
10065,11010402,2016,11,1101,110104,21,18,25,56,32,37,44,45,39,35,26,16,8
10038,9260106,2016,9,926,92601,30,38,26,21,24,31,35,39,42,47,26,9,5
